# Lesing av input datafil (CSV)

#### Viktige variabler/funksjoner:
##### lister:
nodes_data [Node ID, x, y, Fx, Fy, M]
beams_data [Beam ID, Node 1, Node 2, Geo, Fy, M]
punktlast_data = [Beam ID, Node ID, Type, Vinkel, intensitet N]
fordeltlast_data = [Beam ID, Node ID, Type, Vinkel, intensitet N]
geo_data = [Geo nr., B, b, H, h, R, r, t_steg, t_flens, mat_type, E]


##### dictionaries:
Dictionaries hentes ved for eksempel node_lists['Node 12']
node_lists [Node ID, nodes_data] <br>
beam_lists [Beam ID, beams_data] <br>


In [2]:
import numpy as np
import pandas as pd

### Konstantar

In [3]:
flyt = 355*10^6     #Pa

In [7]:
# Lastar dataen frå Excel-fila
df = pd.read_excel('inputfil-eksempel2(1).xlsx')

# Tek ut spesefike verdiar frå datafila
antall_laster = df.iloc[0, 1]
antall_beams = df.iloc[1, 1]
antall_tverrsnitt = df.iloc[1, 3]

print("Antall Laster:", antall_laster)
print("Antall Beams:", antall_beams)
print("Antall Tverrsnitt:", antall_tverrsnitt)

print()
print(df)

Antall Laster: 14
Antall Beams: 23
Antall Tverrsnitt: 5

      Unnamed: 0 Unnamed: 1         Unnamed: 2 Unnamed: 3      Unnamed: 4  \
0   Antall Nodes         14      Antall Laster         13             NaN   
1   Antall Beams         23  Antall tverrsnitt          5  Boundary codes   
2            NaN    Node ID                  x          y              Fx   
3           Node          1                  0          0               1   
4           Node          2                200          0               0   
..           ...        ...                ...        ...             ...   
63             1          0                  0          0               0   
64             3        0.4                  0        0.7               0   
65             2        0.3                0.2        0.6             0.5   
66             4          0                  0          0               0   
67             5          0                  0          0               0   

   Unnamed: 5 Unna

In [31]:
# Definerar tomme listar:
nodes_data = [] 
beams_data = []
punktlast_data = []
fordeltlast_data = []
geo_data = []

# Vel ut alle radar som startar med 'Node' og legger dei i ei liste
for _, row in df.iterrows():                # itererar rad for rad
    if str(row.iloc[0]) == 'Node':          # Bruk .iloc[0] for å få verdien i første kolonne
        nodes_data.append(list(row.iloc[1:7]))  # legg til all data frå den noden i ei liste med listar
# Lagar lister med namn: Node 1, Node 2... og legg dei i eit dictionary
node_lists = {f'Node {i+1}': nodes_data[i] for i in range(len(nodes_data))}

# Gjer det same for beams:
for _, row in df.iterrows():
    if str(row.iloc[0]) == 'Beam':
        beams_data.append(list(row.iloc[1:6]))
beam_lists = {f'Beam {beams_data[i][0]}': beams_data[i] for i in range(len(beams_data))}


""""
print("Beams Data:")
for key, value in beam_lists.items():
    print(f"{key}: {value}")
"""

# Punktlast:
for _, row in df.iterrows():
    if str(row.iloc[0]).startswith('Punktlast'):
        punktlast_data.append(list(row.iloc[1:6]))
punktlast_lists = {f'Punktlast {i+1}': punktlast_data[i] for i in range(len(punktlast_data))}

# Fordeltlast:
for _, row in df.iterrows():
    if str(row.iloc[0]).startswith('Fordeltlast'):
        fordeltlast_data.append(list(row.iloc[1:6]))
fordelt_lists = {f'Fordeltlast {i+1}': fordeltlast_data[i] for i in range(len(fordeltlast_data))}

# Geometridata
geo_found = False
for _, row in df.iterrows():
    if str(row.iloc[0]) == 'Geo':
        geo_found = True  # Finner kvar det står Geo i dokumentet
        continue  # Dermed fortsetter den med å legge listane med geometridataene i lista
    if geo_found:
        geo_data.append(list(row.iloc[0:11]))  # Bruk .iloc for å unngå fremtidige problemer
geo_lists = {f'Geo {geo_data[i][0]}': geo_data[i] for i in range(len(geo_data))}

{'Geo 1': [1, 0, 0, 0, 0, 0.75, 0.65, 0, 0, 2, 210000000000], 'Geo 3': [3, 0.4, 0, 0.7, 0, 0, 0, 0.035, 0.045, 1, 70000000000], 'Geo 2': [2, 0.3, 0.2, 0.6, 0.5, 0, 0, 0, 0, 1, 70000000000], 'Geo 4': [4, 0, 0, 0, 0, 0.6, 0.45, 0, 0, 2, 210000000000], 'Geo 5': [5, 0, 0, 0, 0, 0.65, 0.55, 0, 0, 2, 210000000000]}


### Definerar dei forskjellige geometriane

In [6]:
# Funksjon for andre arealmoment for dei forskjellige geometriane
# GeometriID er dei forskjellige geometridataen 1, 2, 3, 4...

for i in range(len(geo_data)):
    geometri = geo_data[i]
    ID = int(geometri[0])
    
    if ID == 1:
        R = geometri[5]
        r = geometri[6]
        E_1 = geometri[10]          #N/m

        I_1 = np.pi*(R**4-r**4)/4   #m^4
        A_1 = np.pi*(R**2-r**2)     #m^2

    elif ID == 2:
        B = geometri[1]
        b = geometri[2]
        H = geometri[3]
        h = geometri[4]
        E_2 = geometri[10]

        I_2 = 1/12 * (B*H**3 - b*h**3)  #om sterk aske
        A_2 = B*H-b*H
    
    elif ID == 3:
        B = geometri[1]
        H = geometri[3]
        t_steg = geometri[7]
        t_flens = geometri[8]
        E_3 = geometri[10]

        H_tot = H + 2*t_flens
        b = 0.5*(B-t_steg)
        I_3 = (1/12)*(B*H_tot**3 - 2*b*H**3) #om sterk akse
        A_3 = B*H_tot - b*H

    elif ID == 4:
        R = geometri[5]
        r = geometri[6]
        E_4 = geometri[10]
        
        I_4 = np.pi*(R**4-r**4)/4
        A_4 = np.pi*(R**2-r**2)

    elif ID == 5:
        R = geometri[5]
        r = geometri[6]
        E_5 = geometri[10]
        
        I_5 = np.pi*(R**4-r**4)/4
        A_5 = np.pi*(R**2-r**2) 
        


### Elementlengder

In [ ]:
def lengde(knutepunkt, element, nelem):
    elementlengder = np.zeros((nelem, 1))
    # Beregner elementlengder med Pythagoras' læresetning
    for i in range (0, nelem):
        # OBS! Grunnet indekseringsyntaks i Python-arrays vil ikke denne funksjonen fungere naar vi bare har ett element.
        dx = knutepunkt[element[i, 0], 0] - knutepunkt[element[i, 1], 0]
        dy = knutepunkt[element[i, 0], 1] - knutepunkt[element[i, 1], 1]
        elementlengder[i] = np.sqrt(dx*dx + dy*dy)
    return elementlengder



### Stivhetsmatriser
#### elementstivhetsmatriser, fastinnspenningsmoment, lastvektor, oppbygging av systemstivhetsmatrise, beregning av bøyemoment, max normalspenning (bøyning + aksial) osv